In [1]:
import pandas as pd
from src.training.longformer_pipeline import LongFormerPipeline
from src.training.bert_pipeline import TrainingBertPipeline
import logging
import torch
import os

In [2]:
df = pd.read_csv("data/aes_dataset_5k.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5363 entries, 0 to 5362
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   question           4859 non-null   object 
 1   reference_answer   5363 non-null   object 
 2   answer             5363 non-null   object 
 3   score              5363 non-null   float64
 4   dataset            5363 non-null   object 
 5   normalized_score   5363 non-null   float64
 6   normalized_score2  5363 non-null   int64  
 7   bert_length        5363 non-null   int64  
 8   indobert_length    5363 non-null   int64  
 9   albert_length      5363 non-null   int64  
 10  longformer_length  5363 non-null   int64  
 11  multibert_length   5363 non-null   int64  
 12  indoalbert_length  5363 non-null   int64  
dtypes: float64(2), int64(7), object(4)
memory usage: 544.8+ KB


In [3]:
# Check if the first file exists
df_result = None
if os.path.exists("experiments/results/results_longformer.csv"):
    df_result = pd.read_csv("experiments/results/results_longformer.csv")
    print(df_result['config_id'].iloc[-1])
else:
    print("File 'results_longformer.csv' does not exist.")

df_result1 = None
# Check if the second file exists
if os.path.exists("experiments/results/results_epoch_longformer.csv"):
    df_result1 = pd.read_csv("experiments/results/results_epoch_longformer.csv")
    print(min(df_result1['valid_qwk']))
else:
    print("File 'results_epoch_longformer.csv' does not exist.")

0
0.0


In [4]:
results = []
results_epoch = []
batch_sizes = [2]
epochs_list = [1]
learning_rates = [1e-5]
idx = df_result['config_id'].iloc[-1] if df_result is not None and not df_result.empty else 0  # index untuk setiap kombinasi
best_valid_qwk = min(df_result1['valid_qwk']) if df_result1 is not None and not df_result1.empty else float("-inf")
ROOT_DIR = os.getcwd()

In [5]:
for batch_size in batch_sizes:
    for num_epochs in epochs_list:
        for lr in learning_rates:
            config = {
                "df": df,
                "model_name": "allenai/longformer-base-4096",
                "batch_size": batch_size,
                "learning_rate": lr,
                "epochs": num_epochs,
                "config_id": idx,
                "max_seq_len": 2048,
                "best_valid_qwk": min(df_result1['valid_qwk']),
            }

            logging.info(
                f"Running configuration: config_id={idx}, model_name={config['model_name']}, "
                f"batch_size={batch_size}, epochs={num_epochs}, learning_rate={lr}, max_seq={config['max_seq_len']}"
            )
            print(
                f"\nRunning configuration: config_id={idx}, model_name={config['model_name']}, "
                f"batch_size={batch_size}, epochs={num_epochs}, learning_rate={lr}, max_seq={config['max_seq_len']}"
            )
            
            try:
                pipeline = LongFormerPipeline(config, results, results_epoch)
                pipeline.run_training()

                # Save results
                # Dapatkan root project
                results_path = os.path.join(ROOT_DIR, "experiments/results/results_longformer.csv")
                results_epoch_path = os.path.join(ROOT_DIR, "experiments/results/results_epoch_longformer.csv")
                TrainingBertPipeline.save_csv(results, results_path)
                TrainingBertPipeline.save_csv(results_epoch, results_epoch_path)
            except Exception as e:
                logging.error(f"Error in config_id={idx}: {str(e)}")
                print(f"Error in config_id={idx}: {str(e)}")
                torch.cuda.empty_cache()
            finally:
                # Clear GPU memory after every configuration
                del pipeline.model
                del pipeline.tokenizer
                del pipeline.optimizer
                torch.cuda.empty_cache()
            idx += 1


Running configuration: config_id=0, model_name=allenai/longformer-base-4096, batch_size=2, epochs=1, learning_rate=1e-05, max_seq=2048
split dataset run...
create dataset run...
create dataloader run...
create dataloader done...
====== Training Epoch 1/1 ======
tensor(4869.6958, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(6787.2476, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(2345.3721, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(6297.9697, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(9013.9824, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(8999.8770, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(1214.1641, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(3084.1350, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(4946.5972, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(7349.3389, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(5582.6006, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(1127.0144, device='cuda:0', grad_fn=<